In [1]:
# FUNCTION TO RUN HD_DERIVE ON GITBASH
import subprocess, pathlib, tempfile
import json
GIT_BASH = r"C:\Program Files\Git\git-bash.exe"
def run_git_bash_cmd(cmd):
    with tempfile.TemporaryDirectory() as tmpdir:
        target = pathlib.Path(tmpdir) / "output_file"
        dirname, fname = target.parts[-2:]
        sh_target = f"/tmp/{dirname}/{fname}"
        git_bash_cmd = [GIT_BASH, "--hide", "--no-needs-console", "-lc", f"{cmd} > {sh_target}"]
        #print(git_bash_cmd)
        cmd_result = subprocess.run(git_bash_cmd)
        #print(cmd_result)
        #keys = json.loads(cmd_result)
        cmd_result.check_returncode()
        with target:
            result = target.open().read()
        return result

In [2]:
# CODE TO GENERATE WALLET WITH 3 SETS OF KEYS
import subprocess
import json
from constants import *
import os

mnemonic = os.getenv('mnemonic')

#coinlist=[constants.BTC,constants.BTCTEST,constants.ETH]
coinlist=['btc','btc-test','eth']
print(coinlist)
coins={}

for coin in coinlist:
    wallet=run_git_bash_cmd('./hd-wallet-derive.php -g --coin='+str(coin)+' --mnemonic='+str(mnemonic)+' --numderive=3 --cols=index,address,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=jsonpretty')
    coins.update({coin:json.loads(wallet)})
print(json.dumps(coins,indent=1))


['btc', 'btc-test', 'eth']
{
 "btc": [
  {
   "index": 0,
   "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
   "path": "m/44'/0'/0'/0/0",
   "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
   "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
   "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
   "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
   "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
  },
  {
   "index": 1,
   "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
   "path": "m/44'/0'/0'/0/1",
   "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHofUyxYj7zc6",
   "pubkey": "03412c487c7a81706519d3236215f717b9134a39e0447e1a70de08fd65d83ab7e1",
   "pubkeyhash": "3a9aea7796dbbce5a965d1fa4a7cf01cec335e7d",
   "xprv": "xprvA4EtnVRNTKMCq217SWBa6NsqSVyLTWUXtfzvAS59aMAE7oDrH6mcU6PKk5e

In [22]:
# FUNCTION TO EXTRACT SPECIFIC PRIVATE KEY BY INDEX AND COIN TYPE
def get_priv_key(coin_name,coin_index):
    for k1,v1 in coins.items():
        for elements in v1:
            if coin_name==k1 and coin_index==elements['index']:
                return(elements['privkey'])

def get_address(coin_name,coin_index):
    for k1,v1 in coins.items():
        for elements in v1:
            if coin_name==k1 and coin_index==elements['index']:
                return(elements['address'])            

# TEST FUNCTION ON BTC 0 Keys
get_priv_key('btc',0)
get_address('btc',0)

'1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW'

In [11]:
# FUNCTION TO CONVERT PRIVATE KEY TO ACCOUNT
from web3 import Web3
from bit import *
from eth_account import Account
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

def priv_key_to_account(coin_name,coin_priv_key):
    if coin_name=='eth':
        return(Account.privateKeyToAccount(coin_priv_key))
    if coin_name=='btc_test':
        return(PrivateKeyTestnet(priv_key))

# TEST FUNCTION ON ETH 0 and BTC-TEST 1 Keys
priv_key_to_account('eth',get_priv_key('eth',0))
priv_key_to_account('btc-test',get_priv_key('btc-test',1))

In [21]:
print(priv_key_to_account('eth',get_priv_key('eth',0)).address)

0x02194a55DDA1B029F70220D53f099F38c629ab36


In [23]:
# FUNCTION TO CREATE TRANSACTION
def create_tx(coin_name,account,recipient,amount):
    if coin_name=='eth':
        gasEstimate = w3.eth.estimateGas({"from": account.address, "to": recipient, "value": amount})
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }
    if coin_name=='btc_test':
        return(PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, coin_name)]))
# TEST FUNCTION ON ETH 0 and BTC-TEST 1 Keys
create_tx('eth',priv_key_to_account('eth',get_priv_key('eth',0)),get_address('eth',1),1)
create_tx('btc-test',priv_key_to_account('btc-test',get_priv_key('btc-test',0)),get_address('btc-test',1),1)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002683AC88B48>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# FUNCTION TO SEND TRANSACTION
def send_tx(coin_name, account,receipent,amount):
    if coin_name='eth':
        tx = create_raw_tx(account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    if coin_name='btc_test':
        tx = create_raw_tx(account, recipient, amount,'btc')
        key=wif_to_key(get_priv_key('btc',0))
        tx_hex = key.sign_transaction(tx_data)

In [130]:
## ROUGH AREA TO TEST HD-DERIVE

#testcoin=BTC
#wallet=run_git_bash_cmd('./hd-wallet-derive.php -g --coin=btc --mnemonic="blame exit east guard romance vital sibling sad witness bag earn adapt" --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=jsonpretty')
#print(wallet)

In [6]:
from constants import *
print(constants.)

In [4]:
## TEST PRE_FUNDING OF ACCOUNT

from web3 import Web3
from eth_account import Account
from web3.middleware import geth_poa_middleware

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
print(w3.eth.getBalance("0x2222C2EF61816F5D61c74f429E20c4dFF6a474b3"))
print(w3.eth.getBalance("0xD3C2FcF719CbC908f71a04155Cb704B66255A4A6"))

904625697166532776746648320380374280103671755200316906558262375061821325312
904625697166532776746648320380374280103671755200316906558262375061821325312
